In [4]:
from sklearn.datasets import load_digits
digits = load_digits()
x_train,x_test=digits.data[:600], digits.data[:600]
y_train,y_test=digits.target[:600],digits.target[:600] #显然这里有一个很大的问题，训练集不可以和测试集一样
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(600, 64)
(600, 64)
(600,)
(600,)


In [5]:
import numpy as np
from PIL import Image
def img_show(img):
    pil_img=Image.fromarray(np.uint8(img))
    pil_img.show()

In [6]:
img=x_train[0]
label=y_train[0]
print(label)

0


In [7]:
print(img.shape)

(64,)


In [8]:
img=img.reshape(8,8)#把图像形状变成原来的尺寸，即8*8
print(img.shape)

(8, 8)


In [9]:
img_show(img)

In [10]:
def img_show_2(img):
    img=(img/16)*255
    img=img.astype(np.uint8)
    pil_img=Image.fromarray(img)
    pil_img.show()

In [11]:
img_show_2(img)

In [12]:
digits = load_digits()
x_train, x_test = digits.data[:600],digits.data[600:]
y_train,y_test = digits.target[:600],digits.data[600:] #这个应该是digits.target[]
# 注意训练集和测试集的数据需要不一样
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(600, 64)
(1197, 64)
(600,)
(1197, 64)


In [13]:
digits = load_digits()
x_train, x_test = digits.data[600:],digits.data[:600]
y_train,y_test = digits.target[600:],digits.target[:600]
# 注意训练集和测试集的比例最好 7：3， 8：2
print(x_train.shape)
print(x_test.shape) 
print(y_train.shape)
print(y_test.shape)

(1197, 64)
(600, 64)
(1197,)
(600,)


In [14]:
print(x_train[0])
print(y_train[0])

[ 0.  0. 10. 15.  2.  0.  0.  0.  0.  7. 16. 16.  6.  0.  0.  0.  0. 12.
 13. 12.  9.  0.  0.  0.  0.  8.  9. 13.  7.  0.  0.  0.  0.  0.  0. 16.
  5.  0.  0.  0.  0.  0.  6. 15.  1.  0.  0.  0.  0.  0. 16. 14.  4.  5.
  8.  3.  0.  0.  8. 16. 16. 16. 16.  9.]
2


可以发现x的取值范围是[0,16],y的取值范围是[0,9]

In [15]:
#归一化
x_train = x_train /16
x_test = x_test /16
print(x_train[0])

[0.     0.     0.625  0.9375 0.125  0.     0.     0.     0.     0.4375
 1.     1.     0.375  0.     0.     0.     0.     0.75   0.8125 0.75
 0.5625 0.     0.     0.     0.     0.5    0.5625 0.8125 0.4375 0.
 0.     0.     0.     0.     0.     1.     0.3125 0.     0.     0.
 0.     0.     0.375  0.9375 0.0625 0.     0.     0.     0.     0.
 1.     0.875  0.25   0.3125 0.5    0.1875 0.     0.     0.5    1.
 1.     1.     1.     0.5625]


In [17]:
def to_one_hot(labels,num_classes=10):
    one_hot=np.zeros(len(labels),num_classes)
    one_hot[np.arrange(len(labels)),labels]=1
    return one_hot
    #one_hot is a 2D array（一个矩阵简单理解）
y_train_one_hot=to_one_hot(y_train)
y_test_one_hot=to_one_hot(y_test)
print(y_train_one_hot[0])

TypeError: Cannot interpret '10' as a data type

In [16]:
def to_one_hot(labels,num_classes=10):
    one_hot=np.zeros((len(labels),num_classes))
    one_hot[np.arange(len(labels)),labels]=1
    return one_hot
    #one_hot is a 2D array（一个矩阵简单理解）
y_train_one_hot=to_one_hot(y_train)
y_test_one_hot=to_one_hot(y_test)
print(y_train_one_hot[0])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [18]:
#初始化神经网络参数
def init_parameters(input_size,hidden_size,output_size):
    W1=np.random.randn(hidden_size,input_size)*0.01
    # b1=np.zeros(tuple)
    b1=np.zeros((hidden_size,1))
    W2=np.random.randn(output_size,hidden_size)*0.01
    b2=np.zeros((output_size,1))
    return{"W1": W1, "b1": b1, "W2": W2, "b2": b2 }


In [19]:
#设定激活函数
def sigmoid(z):
    return 1/(1+np.exp(-z))
#激活函数的导数
def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

In [20]:
#定义前向传播
def forward_propa(X,parameters):
    W1,b1,W2,b2=parameters["W1"], parameters["b1"], parameters["W2"], parameters["b2"]
    Z1=np.dot(W1,X.T)+b1
    A1=sigmoid(Z1)
    Z2=np.dot(W2,A1)+b2
    A2=sigmoid(Z2)
    return {"Z1":Z1,"A1":A1, "Z2":Z2, "A2":A2}

In [21]:
#定义损失函数
def compute_loss(A2,Y):
    m=Y.shape[0]
    loss=-np.sum(Y*np.log(A2)+(1-Y)*np.log(1-A2))/m
    return np.squeeze(loss) # 返回一个标量


In [28]:
#反向传播
def backward_propa(X,Y,parameters,cache):
    m=X.shape[0]
    W1,W2=parameters["W1"], parameters["W2"]    
    A1,A2=cache["A1"],cache["A2"]

    dZ2=A2-Y.T
    dW2=np.dot(dZ2,A1.T)/m
    db2=np.sum(dZ2,axis=1,keepdims=True)/m

    dZ1=np.dot(W2.T,dZ2)*sigmoid_derivative(cache["Z1"])
    dW1=np.dot(dZ1,X)/m
    db1=np.sum(dZ1,axis=1,keepdims=True)/ m

    return {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}

In [23]:
#使用梯度下降更新参数
def update_parameters(parameters,grads,learning_rate=0.01):
    parameters["W1"]-=learning_rate*grads["dW1"]
    parameters["b1"]-=learning_rate*grads["db1"]
    parameters["W2"]-=learning_rate*grads["dW2"]
    parameters["b2"]-=learning_rate*grads["db2"]
    return parameters

In [24]:
def train_network(X,Y,hidden_size=50,learning_rate=0.01,epochs=1000):
    input_size=X.shape[1]
    output_size=Y.shape[1]
    parameters=init_parameters(input_size,hidden_size,output_size)
    for i in range(epochs):
        #向前传播
        cache=forward_propa(X,parameters)
        #计算损失函数
        loss=compute_loss(cache["A2"],Y)
        #有点像y hat和y的差距
        if i % 100 == 0:
            print(f'Epoch {i} Loss:{loss:.4f}')
        #反向传播
        grads = backward_propa(X,Y,parameters,cache)
        #更新参数
        parameters = update_parameters(parameters,grads,learning_rate)
    return parameters

In [29]:
print("开始训练神经网络...")
trained_parameters = train_network(x_train,y_train_one_hot,hidden_size=50,learning_rate=0.01,epochs=1000)

开始训练神经网络...
Epoch 0 Loss:6.9913
Epoch 100 Loss:3.2995
Epoch 200 Loss:3.2521
Epoch 300 Loss:3.2504
Epoch 400 Loss:3.2502
Epoch 500 Loss:3.2500
Epoch 600 Loss:3.2499
Epoch 700 Loss:3.2497
Epoch 800 Loss:3.2496
Epoch 900 Loss:3.2494


根据报错的显示：在计算损失函数的时候，需要将A2和Y的形状统一一下
很简单其实只需要转置一下,transpose

In [ ]:
#定义损失函数
#edited
def compute_loss(A2,Y):
    m=Y.shape[0]
    A2=A2.T
    # transpose
    loss=-np.sum(Y*np.log(A2)+(1-Y)*np.log(1-A2))/m
    return np.squeeze(loss) # 返回一个标量


In [30]:
print("开始训练神经网络...")
trained_parameters = train_network(x_train,y_train_one_hot,hidden_size=50,learning_rate=0.01,epochs=1000)
print("训练完成！")

开始训练神经网络...
Epoch 0 Loss:6.9692
Epoch 100 Loss:3.2991
Epoch 200 Loss:3.2520
Epoch 300 Loss:3.2503
Epoch 400 Loss:3.2501
Epoch 500 Loss:3.2499
Epoch 600 Loss:3.2498
Epoch 700 Loss:3.2496
Epoch 800 Loss:3.2494
Epoch 900 Loss:3.2493
训练完成！


In [37]:
network={
    "W1":trained_parameters["W1"],
    "b1":trained_parameters["b1"],
    "W2":trained_parameters["W2"],
    "b2":trained_parameters["b2"]
}

In [38]:
print(f"W1 shape:{network['W1'].shape}")
print(f"W2 shape:{network['W2'].shape}")
print(f"b1 shape:{network['b1'].shape}")
print(f"b2 shape:{network['b2'].shape}")

W1 shape:(50, 64)
W2 shape:(10, 50)
b1 shape:(50, 1)
b2 shape:(10, 1)


In [41]:
def predict(network,x):
    W1,W2=network["W1"],network["W2"]
    b1,b2=network["b1"],network["b2"]
    A1=np.dot(W1,x.T)+b1
    Z1=sigmoid(A1)
    A2=np.dot(W2,Z1)+b2
    Z2=sigmoid(A2)
    y=Z2
    return y


In [44]:
#前面已经构建好的测试集,直接在这里使用
#x_test,y_test
accuracy_cnt = 0
for i in range(len(x_test)):
    y=predict(network,x_test[i])
    p=np.argmax(y)#获取概率最高的元素索引
    if p==y_test[i]:
        accuracy_cnt +=1
print(f"测试集的准确率：{float(accuracy_cnt)/len(x_test):.4f}")

测试集的准确率：0.0000


In [45]:
print(y[0])

[0.10347444 0.09533994 0.09417085 0.09737265 0.09668716 0.09922381
 0.09916439 0.09924201 0.09929889 0.09832176 0.09868644 0.09167084
 0.09292741 0.10057751 0.09970084 0.09283661 0.09247394 0.09796214
 0.09292025 0.09818825 0.0960994  0.09494911 0.09410173 0.1032526
 0.09732718 0.09941935 0.09987965 0.09424328 0.09307692 0.09719858
 0.09917164 0.09941474 0.09148671 0.10272291 0.09236246 0.09583682
 0.09847557 0.10133176 0.09628192 0.09921391 0.09814179 0.09872853
 0.09339136 0.09094935 0.10033254 0.09965695 0.10586307 0.1020583
 0.10299404 0.10222393]


In [46]:
print(y.shape)

(10, 50)


In [52]:
print(y_test.shape)

(600,)


In [49]:
print(y_test[0])

0


In [50]:
print(x_test[0])

[0.     0.     0.3125 0.8125 0.5625 0.0625 0.     0.     0.     0.
 0.8125 0.9375 0.625  0.9375 0.3125 0.     0.     0.1875 0.9375 0.125
 0.     0.6875 0.5    0.     0.     0.25   0.75   0.     0.     0.5
 0.5    0.     0.     0.3125 0.5    0.     0.     0.5625 0.5    0.
 0.     0.25   0.6875 0.     0.0625 0.75   0.4375 0.     0.     0.125
 0.875  0.3125 0.625  0.75   0.     0.     0.     0.     0.375  0.8125
 0.625  0.     0.     0.    ]


In [51]:
print(x_test.shape)

(600, 64)


In [53]:
def predict_2(network,x):
    W1,W2=network["W1"],network["W2"]
    b1,b2=network["b1"],network["b2"]
    A1=np.dot(W1,x.T)+b1
    Z1=sigmoid(A1)
    A2=np.dot(W2,Z1)+b2
    Z2=sigmoid(A2)
    y=Z2.T
    return y
#尝试使用单个数据进行试验一下
y0=predict_2(network,x_test[0])
print(y0)


[[0.10385643 0.11037858 0.10505453 0.10920657 0.11179337 0.10914641
  0.1092183  0.1082355  0.10526294 0.10905934]
 [0.09244865 0.09872673 0.09362394 0.09762028 0.1000952  0.09751424
  0.09762048 0.09665692 0.09384185 0.09741796]
 [0.09227277 0.09854661 0.0934476  0.09744124 0.09991427 0.09733454
  0.09744127 0.09647809 0.09366563 0.09723813]
 [0.09280579 0.09909244 0.09398201 0.09798381 0.10046257 0.09787912
  0.09798436 0.09702003 0.09419968 0.0977831 ]
 [0.09720906 0.10359631 0.09839554 0.1024615  0.10498572 0.10237394
  0.10246638 0.10149349 0.09860996 0.10228125]
 [0.09547818 0.10182698 0.09666087 0.10070231 0.10320905 0.10060792
  0.10070547 0.09973575 0.09687664 0.10051388]
 [0.09418468 0.10050382 0.09536434 0.09938687 0.10188021 0.09928745
  0.09938875 0.09842156 0.09558106 0.09919243]
 [0.09789593 0.10429804 0.09908382 0.10315926 0.10569028 0.10307445
  0.10316483 0.10219076 0.09929767 0.10298231]
 [0.09465633 0.10098637 0.09583711 0.09986659 0.10236485 0.099769
  0.09986894 0

In [54]:
print(y0.shape)

(50, 10)


突然想到一个好办法去检测一下到底是which step has the error and mistakes
take the function apart step by step

In [55]:
def predict_3(network,x):
    W1,W2=network["W1"],network["W2"]
    b1,b2=network["b1"],network["b2"]
    A1=np.dot(W1,x.T)+b1
    Z1=sigmoid(A1)
    y=Z1
    return y
#尝试使用单个数据进行试验一下
y=predict_3(network,x_test[0])
print(y.shape)

(50, 50)


In [56]:
print(f"W1 shape:{network['W1'].shape}")
print(f"W2 shape:{network['W2'].shape}")
print(f"b1 shape:{network['b1'].shape}")
print(f"b2 shape:{network['b2'].shape}")

W1 shape:(50, 64)
W2 shape:(10, 50)
b1 shape:(50, 1)
b2 shape:(10, 1)


In [57]:
print(x_test[0].shape)
W1,b1=network["W1"],network["b1"]
A1=np.dot(W1,x_test[0].T)+b1
print(A1.shape)

(64,)
(50, 50)


Solve it! (65,).T -> (64,) 一维数组转置了还是一维数组

In [60]:
sample=x_test[0].reshape(-1,1)  # Reshape to a column vector
print(sample.shape)

(64, 1)


In [61]:
def predict_4(network,x):
    sample=x.reshape(-1,1)
    W1,W2=network["W1"],network["W2"]
    b1,b2=network["b1"],network["b2"]
    A1=np.dot(W1,sample)+b1
    Z1=sigmoid(A1)
    A2=np.dot(W2,Z1)+b2
    Z2=sigmoid(A2)
    y=Z2.T
    return y
#尝试使用单个数据进行试验一下
y=predict_4(network,x_test[0])
print(y.shape)
print(y)

(1, 10)
[[0.0973032  0.103127   0.09801006 0.10222929 0.10478566 0.10212371
  0.10213708 0.10118934 0.09835362 0.10212567]]


In [62]:
print(y_test[0])

0


In [63]:
accuracy_cnt = 0
for i in range(len(x_test)):
    y=predict_4(network,x_test[i])
    p=np.argmax(y)#获取概率最高的元素索引
    if p==y_test[i]:
        accuracy_cnt +=1
print(f"测试集的准确率：{float(accuracy_cnt)/len(x_test):.4f}")

测试集的准确率：0.0950


In [64]:
def predict_batch(X,network):
    X=X.T
    #(样本数，64)->(64，样本数)，现在每一列是一个样本，转置的目的在于方便后续的矩阵运算
    W1,W2=network["W1"],network["W2"]
    b1,b2=network["b1"],network["b2"]
    Z1=np.dot(W1,X)+b1
    A1=sigmoid(Z1)
    Z2=np.dot(W2,A1)+b2
    A2=sigmoid(Z2)

    return np.argmax(A2,axis=0) # 返回每一列就是一个样本的预测结果


In [65]:
test_predictions = predict_batch(x_test,network)
accuracy = np.mean(test_predictions == y_test)
print(f"测试集准确率：{accuracy:.4f}")

测试集准确率：0.0950
